In [1]:
import json
import tqdm
import pandas as pd

filters = []

def check_filter(json_line):
    for filt in filters:
        if filt(json_line):
            return True

    return False

In [2]:
data_submission = pd.read_csv("askdocs_submissions.csv")

/tmp/ipykernel_1743720/1722438381.py:1: DtypeWarning: Columns (0,7,8,9,10,14,15,16,17,18,20,22,24,27,28,30,32,34,35,36,39,40,41,42,43,51,53,55,60,62,63,64,65,66,67,68,69,70,73,74,75,76,79,82,84,86,89,90,92,93,94,96,102,104,105,106,107,108,109,111,112,114,115,116,117,118) have mixed types. Specify dtype option on import or set low_memory=False.
  data_submission = pd.read_csv("askdocs_submissions.csv")


In [3]:
parsed_data = {}
pruned_ids = []

filters = [
    lambda k: not isinstance(k['selftext'], str),
    lambda k: "[removed]" in k['selftext'],
    lambda k: k['banned_at_utc'] > 0,
    lambda k: len(k['selftext']) < 5,
]

for row_id, row in tqdm.tqdm_notebook(data_submission.iterrows()):
    if check_filter(row):
        pruned_ids.append(row['id'])
    else:
        _id = row['id']
        assert _id is not parsed_data

        row['comments'] = []

        parsed_data[_id] = row

/tmp/ipykernel_1743720/214341143.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row_id, row in tqdm.tqdm_notebook(data_submission.iterrows()):


0it [00:00, ?it/s]

In [4]:
len(parsed_data)

597504

In [5]:
len(pruned_ids)

235795

In [6]:
data_comments = pd.read_csv("duplicates_removed.csv")

/tmp/ipykernel_1743720/2725224578.py:1: DtypeWarning: Columns (3,6,8,10,11,12,15,18,20,22,23,24,25,27,29,30,34,35,41,44,46,51,53,57,60,65,71,73,75) have mixed types. Specify dtype option on import or set low_memory=False.
  data_comments = pd.read_csv("duplicates_removed.csv")


In [7]:
class CommentNode:
    def __init__(self, comment):
        self.id = comment['id']
        self.comment = comment
        self.children = []


class CommentTree:
    def __init__(self, root):
        self.root = root

In [8]:
parsed_data_comments = []
pruned_ids_comments = []

filters = [
    lambda k: not isinstance(k['body'], str),
    lambda k: "[removed]" in k['body'],
    lambda k: k['author'] == 'AutoModerator',
    lambda k: len(k['body']) < 5,
]

misses = 0
missesed = []
hits = 0

for row_id, row in tqdm.tqdm_notebook(data_comments.iterrows()):
    if check_filter(row):
        pruned_ids_comments.append(row['id'])
    else:
        try:
            link_type, parent_id = row['link_id'].split("_")
        except:
            missesed.append(row)
            misses += 1
            continue

        if parent_id not in parsed_data:
            missesed.append(row)
            misses += 1
            continue

        parsed_data[parent_id]['comments'].append(row)
        hits += 1

/tmp/ipykernel_1743720/2965173668.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row_id, row in tqdm.tqdm_notebook(data_comments.iterrows()):


0it [00:00, ?it/s]

In [ ]:
len(pruned_ids_comments)

In [9]:
misses

64034

In [10]:
hits

1276396

In [11]:
import dill

data = dill.dump(parsed_data, open("data.pkl", "wb+"))

In [1]:
import dill

data = dill.load(open("data.pkl", "rb"))

In [2]:
list(data.keys())[-99]

'2t21wk'

In [3]:
data['2t21wk']['comments']

[]

In [4]:
comments = list(filter(lambda k: len(k['comments']) > 0, data.values()))
len(comments)

272529

In [5]:
num_comments = list(filter(lambda k: isinstance(k['num_comments'], int) and int(k['num_comments']) > 1, data.values()))
len(num_comments)

304551

In [75]:
# Setup fast search hashmap
import pandas as pd
import tqdm

id_normalize = {}

df = pd.read_csv("PMC-ids.csv", dtype=str)

for _, row in tqdm.tqdm_notebook(df.iterrows(), total=len(df)):
    doi, pmcid, pmid, manuscript_id = list(map(lambda k: str(k), [row['DOI'], row['PMCID'], row['PMID'], row['Manuscript Id']]))
    
    id_normalize[doi] = pmcid
    id_normalize[pmid] = pmcid
    id_normalize[manuscript_id] = pmcid
    id_normalize[pmcid] = pmid

/tmp/ipykernel_11611/3423019674.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _, row in tqdm.tqdm_notebook(df.iterrows(), total=len(df)):


  0%|          | 0/8565178 [00:00<?, ?it/s]

In [50]:
import nltk

hits = 0

def find_and_normalize(s):
    global hits
    ids = []
    tokens = nltk.tokenize.word_tokenize(s)

    for token in tokens:
        if token in id_normalize:
            pmcid = id_normalize[token].strip()
            pmid = id_normalize[pmcid].strip()

            if pmcid:
                ids.append(pmcid)

            if pmid:
                ids.append(pmid)

            hits += 1

    return ids


def pubmed_url_parse():
    pass

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

In [ ]:
counts = 0
lines = []

for document in tqdm.notebook.tqdm(comments):
    for i, comment in enumerate(document['comments']):
        flag = False
        if ("www" in comment['body'] or "http" in comment['body']) and ("pubmed" in comment['body'] or "ncbi" in comment['body']):
            counts += 1
            flag = True

        ids = find_and_normalize(comment['body'])

        if len(ids) > 0:
            flag = True

            document['comments'][i]['pubmed_ids'] = ids
            document['has_pubmed'] = True

        if flag:
            lines.append(document)

In [39]:
len(lines)

3158

In [40]:
dill.dump(lines, open("websites.pkl", "wb+"))

In [98]:
import dill

websites = dill.load(open("websites.pkl", "rb"))

In [4]:
matches

[('https://www.ncbi.nlm.nih.gov/pubmed/18466657', '', '', '', ''),
 ('http://mentalhealthfood.net/the-power-of-potassium/', '', '', '', ''),
 ('https://s-media-cache-ak0.pinimg.com/originals/e8/fc/6b/e8fc6b29d6cfaba14de94c34f72c6841.jpg',
  '',
  '',
  '',
  '')]

In [99]:
def recurse_unwrap(collection, unwound_list):
    if isinstance(collection, (list, tuple)):
        for element in collection:
            unwound_list.append(recurse_unwrap(element, unwound_list))
    else:
        return collection

In [10]:
import re 

pattern = re.compile(r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))""")

lines = websites
matches = pattern.findall(lines[-1]['comments'][6]['body'])

test = []
recurse_unwrap(matches, test)
list(filter(lambda k: k and ("pubmed" in k or "ncbi" in k), test))

['https://www.ncbi.nlm.nih.gov/pubmed/18466657']

In [8]:
import re

pattern = re.compile(r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))""")

In [100]:
import tqdm.notebook

counts = 0
lines = []
found = []

for document in tqdm.notebook.tqdm(websites):
    for i, comment in enumerate(document['comments']):
        matches = pattern.findall(comment['body'])
        urls = []

        recurse_unwrap(matches, urls)
        urls = list(filter(lambda k: k and ("pubmed" in k or "ncbi" in k), urls))
        found.append(urls)

        if len(urls) > 0:
            counts = counts + 1

        if 'urls' in document:
            document['urls'].extend(urls)
        else:
            document['urls'] = urls

  0%|          | 0/2560 [00:00<?, ?it/s]

AttributeError: 'set' object has no attribute 'extend'

In [36]:
unique = {}

for document in websites:
    if document['id'] in unique:
        continue
    else:
        unique[document['id']] = document

In [96]:
websites = list(unique.values())
#dill.dump(websites, open("websites_filtered.pkl", "wb+"))

In [121]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

hits = []
misses = []

for document in websites:
    for comment in document['comments']:
        if 'pmc_ids' not in document:
            document['pmc_ids'] = []
        if 'pmid' not in document:
            document['pmid'] = []
        if 'pubmed_ids' in comment:
            document['pmc_ids'].extend(comment['pubmed_ids'])

    for url in document['urls']:
        ids = []
        pmids = []
        for token in url.split('/'):
            if has_numbers(token):
                token = str(token).strip()
                if token in id_normalize:
                    pmcid = str(id_normalize[token]).strip()

                    if pmcid:
                        ids.append(pmcid)

                    hits.append(url)
                    break
                else:
                    pmid = str(token)
                    document['pmid'].append(pmid)
                    break

        if not ids:
            misses.append(url)

        document['pmc_ids'].extend(ids)

    document['pmc_ids'] = list(set(document['pmc_ids']))
    document['pmid'] = list(set(document['pmid']))

In [122]:
list(websites)[-3]['pmc_ids']

['PMC3889339', '24427724']

In [123]:
dill.dump(list(websites), open("websites_filtered2.pkl", "wb+"))

In [124]:
len(misses)

1644

In [125]:
len(hits)

1785

In [ ]:
from pymetamap import MetaMap

METAMAP_PATH = "/home/vin/Downloads/metamap/public_mm/bin/metamap20"

mm = MetaMap.get_instance(METAMAP_PATH)

In [ ]:
import tqdm

for document in tqdm.tqdm_notebook(websites):
    selftext_mmc = mm.extract_concepts(document['selftext'])
    document['selftext_mmc'] = selftext_mmc

    for i, comment in enumerate(document['comments']):
        comment_body_mmc = mm.extract_concepts([comment['body']])
        comment['body_mmc'] = comment_body_mmc